In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.tabular.all import *
from wwf.tab.export import *
from bnbsl import *

from classes import *
from utils import *

In [3]:
SEASON = '22'
set_seed(int(SEASON))

## Download HKJC odds

In [4]:
path_raw = Path('raw_data')
path_data = Path('data')

path_output = Path('output')

In [5]:
session = requests.Session()
r = session.get('http://bet.hkjc.com')
cookies = r.cookies

In [6]:
odds_url = 'https://bet.hkjc.com/football/getJSON.aspx?jsontype=odds_chl.aspx'
response = session.post(
    odds_url,
    headers={'referer':'http://bet.hkjc.com'},
    cookies=cookies
)

In [7]:
with open(path_data/'json'/f'odds_chl-{datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}.txt', 'w') as f:
    f.write(response.text)

In [8]:
matches_json = json.loads(response.text)['matches']

In [9]:
# Example
# matches_json = json.loads(open(path_data/'json'/'odds_chl-2021-11-20-20-15-14.txt').read())[1]['matches']

In [10]:
matches = [Match(m) for m in matches_json if m['matchStatus'] == 'Defined']
odds = [m.export() for m in matches]

['20230218SATFB7454', '2023-Feb-18', '01:30:00', 'German Division 2 [GD2]', 'Braunschweig', 'Holstein Kiel', 'true', '10.5', '1.94', '1.76', 'false', '11.5', '2.48', '1.47', 'false', '13.5', '4.15', '1.18']
['20230218SATFB7455', '2023-Feb-18', '01:30:00', 'German Division 2 [GD2]', 'Paderborn', 'Kaiserslautern', 'true', '9.5', '1.73', '1.98', 'false', '10.5', '2.17', '1.61', 'false', '13.5', '4.90', '1.13']
['20230218SATFB7453', '2023-Feb-18', '03:30:00', 'German Division 1 [GSL]', 'Augsburg', 'Hoffenheim', 'true', '9.5', '1.83', '1.87', 'false', '10.5', '2.30', '1.54', 'false', '12.5', '3.90', '1.20']
['20230218SATFB7459', '2023-Feb-18', '03:45:00', 'Italian Division 1 [ISA]', 'Sassuolo', 'Napoli', 'true', '9.5', '1.81', '1.89', 'false', '10.5', '2.29', '1.55', 'false', '13.5', '5.30', '1.11']
['20230218SATFB7463', '2023-Feb-18', '04:00:00', 'Spanish Division 1 [SFL]', 'Girona', 'Almeria', 'true', '9.5', '2.02', '1.70', 'false', '10.5', '2.55', '1.45', 'false', '12.5', '4.75', '1.14']

In [11]:
cols_match = ['MatchDay', 'Date', 'Time', 'LeagueJC', 'HomeTeamJC', 'AwayTeamJC']
cols_odds0 = ['MAINLINE_0', 'CHL_LINE_0', 'CHL_H_0', 'CHL_L_0']
cols_odds1 = ['MAINLINE_1', 'CHL_LINE_1', 'CHL_H_1', 'CHL_L_1']
cols_odds2 = ['MAINLINE_2', 'CHL_LINE_2', 'CHL_H_2', 'CHL_L_2']
cols_odds  = ['MAINLINE', 'CHL_LINE', 'CHL_H', 'CHL_L']
cols_pred  = ['alpha_1', 'alpha_2', 'mu_1', 'mu_2', 'omega', 'corr']

cols = cols_match + cols_odds0 + cols_odds1 + cols_odds2

In [12]:
odds = pd.DataFrame(odds, columns=cols)
odds = odds.fillna(value=np.nan)

In [13]:
cols_odds_ = cols_odds0[1:]+cols_odds1[1:]+cols_odds2[1:]
odds[cols_odds_] = odds[cols_odds_].astype(float)

In [14]:
odds

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE_0,CHL_LINE_0,CHL_H_0,CHL_L_0,MAINLINE_1,CHL_LINE_1,CHL_H_1,CHL_L_1,MAINLINE_2,CHL_LINE_2,CHL_H_2,CHL_L_2
0,20230218SATFB7454,2023-Feb-18,01:30:00,German Division 2 [GD2],Braunschweig,Holstein Kiel,true,10.5,1.94,1.76,false,11.5,2.48,1.47,false,13.5,4.15,1.18
1,20230218SATFB7455,2023-Feb-18,01:30:00,German Division 2 [GD2],Paderborn,Kaiserslautern,true,9.5,1.73,1.98,false,10.5,2.17,1.61,false,13.5,4.90,1.13
2,20230218SATFB7453,2023-Feb-18,03:30:00,German Division 1 [GSL],Augsburg,Hoffenheim,true,9.5,1.83,1.87,false,10.5,2.30,1.54,false,12.5,3.90,1.20
3,20230218SATFB7459,2023-Feb-18,03:45:00,Italian Division 1 [ISA],Sassuolo,Napoli,true,9.5,1.81,1.89,false,10.5,2.29,1.55,false,13.5,5.30,1.11
4,20230218SATFB7463,2023-Feb-18,04:00:00,Spanish Division 1 [SFL],Girona,Almeria,true,9.5,2.02,1.70,false,10.5,2.55,1.45,false,12.5,4.75,1.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,20230220MONFB7557,2023-Feb-20,04:00:00,Spanish Division 1 [SFL],Barcelona,Cadiz,true,9.5,1.81,1.89,false,10.5,2.29,1.55,false,12.5,3.80,1.21
78,20230220MONFB7587,2023-Feb-20,09:05:00,Mexican Premier [MXL],CF America,Tijuana,true,10.5,2.08,1.66,false,11.5,2.68,1.41,false,13.5,4.55,1.15
79,20230220MONFB7586,2023-Feb-20,11:05:00,Mexican Premier [MXL],Pachuca,Toluca,true,10.5,1.90,1.80,false,11.5,2.40,1.50,false,13.5,3.90,1.20
80,20230221TUEFB7517,2023-Feb-21,03:45:00,Italian Division 1 [ISA],AC Torino,Cremonese,true,9.5,1.95,1.75,false,10.5,2.55,1.45,false,12.5,4.40,1.16


In [15]:
odds['MatchDay'] = odds['MatchDay'].str[8:]
odds['Date'] = pd.to_datetime(odds['Date'])
odds['Time'] = pd.to_datetime(odds['Time'], format='%H:%M:%S').dt.time

In [16]:
odds['DateTimeJC'] = pd.to_datetime(odds['Date'].dt.date.map(str) + '-' + odds['Time'].map(str))
odds['DateTimeJC'] = odds['DateTimeJC'].dt.tz_localize('Hongkong')
odds['DateTime'] = odds['DateTimeJC'].dt.tz_convert('GB')

In [17]:
map_league = pd.read_csv(path_data/'league.csv')
map_team = pd.read_csv(path_data/'team.csv')

In [18]:
div = ['E0', 'E1', 'E2', 'D1', 'D2', 'SP1', 'SP2', 'I1', 'I2', 'F1', 'F2', 'UCL', 'UEC', 'UEF']
map_league = map_league[map_league.Div.isin(div)].reset_index(drop=True)

In [19]:
# Map Div name
odds = odds.merge(map_league[['LeagueJC', 'Div']], 'inner', on='LeagueJC')

In [20]:
# Map Team name
odds = odds.merge(map_team[['TeamNameJC', 'TeamName']].rename(columns={'TeamName':'HomeTeam'}), 'inner', 
                  left_on='HomeTeamJC', right_on='TeamNameJC').drop(columns=['TeamNameJC'])
odds = odds.merge(map_team[['TeamNameJC', 'TeamName']].rename(columns={'TeamName':'AwayTeam'}), 'inner', 
                  left_on='AwayTeamJC', right_on='TeamNameJC').drop(columns=['TeamNameJC'])

## Download recent stats

In [21]:
# Download latest results in current season
!wget -q https://www.football-data.co.uk/mmz4281/{SEASON}{int(SEASON)+1}/data.zip -O raw_data/data.zip

# Unzip to folder
!unzip -q -o raw_data/data.zip -d raw_data/{SEASON}

In [22]:
usecols = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'HC', 'AC', 'FTHG', 'FTAG', 'HS', 'AS', 'HST', 'AST']
dtype = {'HC':'float', 'AC':'float'}
parse_dates = ['Date']

seasons = [SEASON]

dfs = []

for folder in sorted(path_raw.iterdir()):
    if folder.is_dir() and folder.name in seasons: 
        for file in sorted(folder.glob('*.csv')):
            try:
                df = pd.read_csv(file, usecols=usecols, dtype=dtype, parse_dates=parse_dates, dayfirst=True)
                df['Season'] = folder.name
                dfs.append(df)
            except:
                continue

In [23]:
df_season = pd.concat(dfs)
df_season = df_season.dropna()
df_season = df_season.sort_values(['Div', 'Date', 'HomeTeam']).reset_index(drop=True)

In [24]:
df_hist = pd.read_csv(path_data/'data.csv', dtype={'HC':'float', 'AC':'float'}, parse_dates=['Date'])
df_hist = df_hist.query(f'Season == {int(SEASON)-1}').reset_index(drop=True)

In [25]:
df_season = pd.concat([df_hist[df_season.columns], df_season])

In [26]:
# Make features on historical stats (Home and Away)
stats = ['FTHG', 'HS', 'HST', 'HC', 'FTAG', 'AS', 'AST', 'AC']
df_home, df_away = joinLastGamesStatsHomeAway(df_season, stats)

In [27]:
# Make features on historical stats (For and Against)
stats = [('FTHG', 'FTAG', 'FTG'), ('HS', 'AS', 'S'), ('HST', 'AST', 'ST'), ('HC', 'AC', 'C')]
df_for, df_against = joinLastGamesStatsForAgainst(df_season, stats)

In [28]:
df_home = df_home.sort_values(['HomeTeam', 'Date']).reset_index(drop=True)
df_away = df_away.sort_values(['AwayTeam', 'Date']).reset_index(drop=True)

df_home = df_home.groupby('HomeTeam')[df_home.columns[df_home.columns.str.contains('Avg')]].last().reset_index()
df_away = df_away.groupby('AwayTeam')[df_away.columns[df_away.columns.str.contains('Avg')]].last().reset_index()

In [29]:
odds = odds.merge(df_home, 'left', 'HomeTeam').merge(df_away, 'left', 'AwayTeam')

In [30]:
cols_home = df_for.columns[df_for.columns.str.contains('Avg')]
cols_home = dict(zip(cols_home, 'Home'+cols_home))
cols_home.update({'Team':'HomeTeam'})

cols_away = df_for.columns[df_for.columns.str.contains('Avg')]
cols_away = dict(zip(cols_home, 'Away'+cols_away))
cols_away.update({'Team':'AwayTeam'})

df_for = df_for.groupby('Team')[df_for.columns[df_for.columns.str.contains('Avg')]].last().reset_index()
odds = odds.merge(df_for.rename(columns=cols_home), 'left', 'HomeTeam').merge(df_for.rename(columns=cols_away), 'left', 'AwayTeam')

In [31]:
cols_home = df_against.columns[df_against.columns.str.contains('Avg')]
cols_home = dict(zip(cols_home, 'Home'+cols_home))
cols_home.update({'Team':'HomeTeam'})

cols_away = df_against.columns[df_against.columns.str.contains('Avg')]
cols_away = dict(zip(cols_home, 'Away'+cols_away))
cols_away.update({'Team':'AwayTeam'})

df_against = df_against.groupby('Team')[df_against.columns[df_against.columns.str.contains('Avg')]].last().reset_index()
odds = odds.merge(df_against.rename(columns=cols_home), 'left', 'HomeTeam').merge(df_against.rename(columns=cols_away), 'left', 'AwayTeam')

In [32]:
add_datepart(odds, 'DateTime', prefix='', drop=False);

In [33]:
display_df(odds.head(5).T)

,0,1,2,3,4
MatchDay,SATFB7454,SATFB7455,SATFB7503,SATFB7502,SATFB7501
Date,2023-02-18 00:00:00,2023-02-18 00:00:00,2023-02-18 00:00:00,2023-02-18 00:00:00,2023-02-18 00:00:00
Time,01:30:00,01:30:00,20:00:00,20:00:00,20:00:00
LeagueJC,German Division 2 [GD2],German Division 2 [GD2],German Division 2 [GD2],German Division 2 [GD2],German Division 2 [GD2]
HomeTeamJC,Braunschweig,Paderborn,Greuther Furth,Jahn Regensburg,Magdeburg
AwayTeamJC,Holstein Kiel,Kaiserslautern,Dusseldorf,Hannover,St. Pauli
MAINLINE_0,true,true,true,true,true
CHL_LINE_0,10.5,9.5,9.5,9.5,9.5
CHL_H_0,1.94,1.73,1.81,1.89,1.84
CHL_L_0,1.76,1.98,1.89,1.81,1.86


## Load model

In [34]:
learn_bnb = load_learner('models/learn_bnbsl.pkl')

In [35]:
to = load_pandas('models/to.pkl')

In [36]:
def predict(self, row):
    "Predict on a Pandas Series"
    dl = self.dls.test_dl(row.to_frame().T)
    dl.dataset.conts = dl.dataset.conts.astype(np.float32)
    inp,preds,_ = self.get_preds(dl=dl, with_input=True, with_decoded=False)
    b = tuplify(inp)
    full_dec = self.dls.decode(b)
    return full_dec,preds[0]

learn_bnb.predict = MethodType(predict, learn_bnb)

In [37]:
odds = odds.dropna(how='any')

In [38]:
to_tst = to.new(odds)
to_tst.process()
# to_tst.items.head()

In [39]:
tst_dl = learn_bnb.dls.valid.new(to_tst)
tst_dl.show(max_n=999)

,Div,HomeTeam,AwayTeam,HSLast5Avg,HCLast5Avg,ASLast5Avg,ACLast5Avg,HomeSForLast5Avg,AwaySForLast5Avg,HomeSAgainstLast5Avg,AwaySAgainstLast5Avg
0,D2,Braunschweig,Holstein Kiel,12.6,6.0,18.6,5.6,9.8,16.2,12.8,14.0
1,D2,Paderborn,Kaiserslautern,16.8,6.8,11.6,5.0,13.8,10.8,14.8,15.2
2,D2,Greuther Furth,Fortuna Dusseldorf,13.6,6.4,13.0,4.8,11.4,17.0,10.8,12.8
3,D2,Regensburg,Hannover,12.8,5.8,11.0,4.4,12.2,10.0,13.4,17.4
4,D2,Magdeburg,St Pauli,12.2,4.8,14.0,5.6,16.4,14.4,16.4,14.4
5,D2,Hansa Rostock,Darmstadt,14.4,6.2,13.4,3.2,12.4,15.0,14.4,9.6
6,D2,Hamburg,Bielefeld,19.4,7.2,12.4,3.2,17.6,17.0,10.0,12.4
7,D2,Heidenheim,Nurnberg,13.0,5.2,9.2,4.4,14.2,14.0,10.6,17.0
8,D2,Sandhausen,Karlsruhe,15.2,5.8,13.0,4.2,12.2,13.0,16.2,15.2
9,D1,Augsburg,Hoffenheim,14.2,5.2,13.0,4.8,14.6,14.0,14.4,12.8


In [40]:
pred, _ = learn_bnb.get_preds(dl=tst_dl)

In [41]:
alpha = F.softplus(pred[:, 0:2])
mu = F.softplus(pred[:, 2:4])
omega = pred[:, 4]

total_count = 1. / alpha
logits = torch.log(alpha * mu)

# print(alpha, mu, omega)

In [42]:
# Method 1:
corr = omega * torch.sqrt(mu.prod(-1)) * alpha.prod(-1) / ((1.+alpha)**(1./alpha+1.)).prod(-1) / torch.sqrt((1.+alpha*mu).prod(-1))

In [43]:
# Method 2:
# d = 1.-math.exp(-1)
# corr = omega * d**2 * torch.sqrt(mu.prod(-1)*(1.+alpha*mu).prod(-1)) * ((1.+d*alpha*mu)**(-1-1/alpha)).prod(-1)

In [44]:
odds[cols_pred] = torch.cat([alpha, mu, omega.unsqueeze(-1), corr.unsqueeze(-1)], dim=-1)

In [45]:
odds0 = odds[cols_match+cols_odds0+cols_pred].rename(columns=dict(zip(cols_odds0, cols_odds)))
odds1 = odds[cols_match+cols_odds1+cols_pred].rename(columns=dict(zip(cols_odds1, cols_odds)))
odds2 = odds[cols_match+cols_odds2+cols_pred].rename(columns=dict(zip(cols_odds2, cols_odds)))

In [46]:
odds = pd.concat([odds0, odds1, odds2]).dropna().reset_index(drop=True)
odds['MAINLINE'] = np.where(odds['MAINLINE']=='true', True, False)

In [47]:
odds.head(5)

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE,CHL_LINE,CHL_H,CHL_L,alpha_1,alpha_2,mu_1,mu_2,omega,corr
0,SATFB7454,2023-02-18,01:30:00,German Division 2 [GD2],Braunschweig,Holstein Kiel,True,10.5,1.94,1.76,0.064370,0.057900,5.015067,5.534815,-49.421875,-0.093593
1,SATFB7455,2023-02-18,01:30:00,German Division 2 [GD2],Paderborn,Kaiserslautern,True,9.5,1.73,1.98,0.072546,0.062049,4.714330,5.767746,-49.374565,-0.108787
2,SATFB7503,2023-02-18,20:00:00,German Division 2 [GD2],Greuther Furth,Dusseldorf,True,9.5,1.81,1.89,0.050879,0.077798,5.878744,4.962787,-51.414295,-0.104100
3,SATFB7502,2023-02-18,20:00:00,German Division 2 [GD2],Jahn Regensburg,Hannover,True,9.5,1.89,1.81,0.059032,0.068233,5.429016,4.995702,-48.829086,-0.097882
4,SATFB7501,2023-02-18,20:00:00,German Division 2 [GD2],Magdeburg,St. Pauli,True,9.5,1.84,1.86,0.069475,0.109391,5.481087,4.290027,-45.929893,-0.147462


In [48]:
odds['corr'].hist(grid=False);

In [49]:
prob_hilo = []

for r in list(zip(odds['alpha_1'], odds['alpha_2'], odds['mu_1'], odds['mu_2'], odds['omega'], odds['CHL_LINE'])):
    total_count = 1. / torch.tensor(r[0:2], device='cpu') 
    logits = torch.log(torch.tensor(r[0:2], device='cpu') * torch.tensor(r[2:4], device='cpu'))
    omega = torch.tensor(r[4], device='cpu')

    bnb_corner = BivariateNegativeBinomialSL(total_count=total_count, omega=omega, logits=logits)
    value = torch.cartesian_prod(torch.arange(0., 15.), torch.arange(0., 15.))
    corner = bnb_corner.log_prob(value).exp()
    
    line = r[5]
    mask = value.sum(-1) < line
    prob_lo = corner[mask].sum()
    prob_hi = 1 - prob_lo
    
    prob_hilo.append([prob_hi.item(), prob_lo.item()])

/Users/twtang/miniconda3/envs/fastai2/lib/python3.8/site-packages/torch/functional.py:1069: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1634272482218/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.cartesian_prod(tensors)  # type: ignore[attr-defined]


In [50]:
odds[['prob_hi', 'prob_lo']] = prob_hilo

In [51]:
odds['kelly_hi'] = (odds['prob_hi'] * odds['CHL_H'] - 1) / (odds['CHL_H'] - 1)
odds['kelly_lo'] = (odds['prob_lo'] * odds['CHL_L'] - 1) / (odds['CHL_L'] - 1)

In [52]:
odds['kelly'] = np.where(
    np.maximum(odds['kelly_hi'], odds['kelly_lo']) > 0, 
    np.where(odds['kelly_hi'] > odds['kelly_lo'], odds['kelly_hi'], odds['kelly_lo']), 
    np.nan
)

In [53]:
odds['bet'] = np.where(
    np.maximum(odds['kelly_hi'], odds['kelly_lo']) > 0, 
    np.where(odds['kelly_hi'] > odds['kelly_lo'], 'High', 'Low'), 
    None
)

In [54]:
odds = odds.sort_values('kelly', ascending=False).reset_index(drop=True)

In [55]:
odds['selected'] = np.where(
    odds['MAINLINE']==True, np.where(
        odds['kelly']>0.3, '$$$', np.where(
            odds['kelly']>0.2, '$$', np.where(
                odds['kelly']>0.1, '$', None))), 
    None
)

In [56]:
odds = odds.drop(columns=cols_pred+['kelly_hi', 'kelly_lo'])

In [57]:
odds[odds.bet.notna() & odds.selected.notna()]

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE,CHL_LINE,CHL_H,CHL_L,prob_hi,prob_lo,kelly,bet,selected
1,SUNFB7553,2023-02-19,20:30:00,German Division 2 [GD2],Heidenheim,Nurnberg,True,9.5,1.83,1.87,0.691748,0.308252,0.320360,High,$$$
2,MONFB7610,2023-02-20,03:45:00,Italian Division 1 [ISA],Roma,Verona,True,9.5,1.86,1.84,0.665713,0.334287,0.277007,High,$$
7,MONFB7499,2023-02-20,00:30:00,German Division 1 [GSL],Dortmund,Hertha Berlin,True,10.5,2.05,1.68,0.307358,0.692642,0.240645,Low,$$
10,SATFB7516,2023-02-18,22:00:00,Italian Division 1 [ISA],Sampdoria,Bologna,True,9.5,2.01,1.71,0.612402,0.387598,0.228641,High,$$
11,SATFB7463,2023-02-18,04:00:00,Spanish Division 1 [SFL],Girona,Almeria,True,9.5,2.02,1.70,0.322146,0.677854,0.217646,Low,$$
16,SUNFB7525,2023-02-19,04:00:00,Spanish Division 1 [SFL],Osasuna,Real Madrid,True,9.5,1.83,1.87,0.382348,0.617652,0.178172,Low,$
17,SUNFB7523,2023-02-19,20:00:00,French Division 1 [FFL],Paris Saint Germain,Lille,True,8.5,1.60,2.19,0.691688,0.308312,0.177835,High,$
19,SATFB7477,2023-02-18,23:00:00,Eng Premier [EPL],Everton,Leeds,True,9.5,1.83,1.87,0.622759,0.377241,0.168251,High,$
22,SATFB7503,2023-02-18,20:00:00,German Division 2 [GD2],Greuther Furth,Dusseldorf,True,9.5,1.81,1.89,0.622681,0.377319,0.156854,High,$
23,SATFB7530,2023-02-18,23:15:00,Spanish Division 1 [SFL],Betis,Valladolid,True,9.5,1.90,1.80,0.599271,0.400729,0.154016,High,$


In [58]:
display_df(odds[odds.MatchDay.isin(odds[odds.bet.notna() & odds.selected.notna()].MatchDay) & odds.bet.notna()])

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE,CHL_LINE,CHL_H,CHL_L,prob_hi,prob_lo,kelly,bet,selected
0,MONFB7499,2023-02-20,00:30:00,German Division 1 [GSL],Dortmund,Hertha Berlin,False,13.5,4.30,1.17,0.095804,0.904196,0.340645,Low,None
1,SUNFB7553,2023-02-19,20:30:00,German Division 2 [GD2],Heidenheim,Nurnberg,True,9.5,1.83,1.87,0.691748,0.308252,0.320360,High,$$$
2,MONFB7610,2023-02-20,03:45:00,Italian Division 1 [ISA],Roma,Verona,True,9.5,1.86,1.84,0.665713,0.334287,0.277007,High,$$
3,SATFB7463,2023-02-18,04:00:00,Spanish Division 1 [SFL],Girona,Almeria,False,10.5,2.55,1.45,0.225928,0.774072,0.272011,Low,None
4,SUNFB7553,2023-02-19,20:30:00,German Division 2 [GD2],Heidenheim,Nurnberg,False,10.5,2.30,1.54,0.585788,0.414212,0.267164,High,None
5,MONFB7499,2023-02-20,00:30:00,German Division 1 [GSL],Dortmund,Hertha Berlin,False,11.5,2.65,1.42,0.217673,0.782327,0.264058,Low,None
6,SUNFB7525,2023-02-19,04:00:00,Spanish Division 1 [SFL],Osasuna,Real Madrid,False,12.5,3.80,1.21,0.127820,0.872180,0.263511,Low,None
7,MONFB7499,2023-02-20,00:30:00,German Division 1 [GSL],Dortmund,Hertha Berlin,True,10.5,2.05,1.68,0.307358,0.692642,0.240645,Low,$$
8,MONFB7610,2023-02-20,03:45:00,Italian Division 1 [ISA],Roma,Verona,False,10.5,2.38,1.51,0.557283,0.442717,0.236473,High,None
9,SUNFB7525,2023-02-19,04:00:00,Spanish Division 1 [SFL],Osasuna,Real Madrid,False,10.5,2.25,1.57,0.277973,0.722027,0.234356,Low,None


In [59]:
odds[odds.MAINLINE].to_csv(path_output/f'odds-{datetime.now().strftime("%Y-%m-%d")}.csv', float_format='%.2f', index=False)